# 📊 Dashboard de Inteligencia de Negocios - Análisis de Ventas Honduras 2025

## Objetivo
Crear un panel de control interactivo que permita visualizar la salud del negocio de un vistazo en el contexto hondureño, respondiendo preguntas clave como:
- ¿Cuánto hemos vendido este mes/trimestre/año? (en Lempiras)
- ¿Cuáles son nuestros 5 productos más vendidos?
- ¿Quiénes son nuestros clientes más valiosos?
- ¿Qué días de la semana se vende más en Honduras?

---

## 1. Cargar y Configurar Librerías

In [1]:
# Importar librerías necesarias para análisis de datos y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings

# Configuración de visualización
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Configuración de pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías cargadas exitosamente!")

✅ Librerías cargadas exitosamente!


## 2. Cargar y Explorar Datos

In [3]:
# Cargar el archivo CSV de ventas
try:
    # Intentar diferentes encodings para manejar caracteres especiales
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
    df = None
    
    for encoding in encodings:
        try:
            df = pd.read_csv('VENTAS 2025.CSV', encoding=encoding)
            print(f"✅ Archivo cargado exitosamente con encoding {encoding}: {len(df)} filas")
            break
        except UnicodeDecodeError:
            continue
    
    if df is None:
        raise ValueError("No se pudo cargar el archivo con ninguna codificación")
    
    print(f"📊 Columnas disponibles: {list(df.columns)}")
    print("\n" + "="*50)
    print("🔍 EXPLORACIÓN INICIAL DE DATOS")
    print("="*50)
except FileNotFoundError:
    print("❌ Error: No se pudo encontrar el archivo 'VENTAS 2025.CSV'")
    print("Asegúrate de que el archivo esté en el directorio correcto.")
except Exception as e:
    print(f"❌ Error al cargar el archivo: {e}")
    print("Intentando con encoding alternativo...")

# Mostrar información básica del dataset
print("\n📋 Información general del dataset:")
df.info()

print("\n📈 Primeras 10 filas:")
df.head(10)

✅ Archivo cargado exitosamente con encoding latin-1: 32768 filas
📊 Columnas disponibles: ['Unnamed: 0', 'Type', 'Date', 'Num', 'Memo', 'Name', 'Item', 'Qty', 'Sales Price', 'Amount', 'Balance']

🔍 EXPLORACIÓN INICIAL DE DATOS

📋 Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32768 entries, 0 to 32767
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1357 non-null   object 
 1   Type         31411 non-null  object 
 2   Date         31411 non-null  object 
 3   Num          31411 non-null  float64
 4   Memo         31411 non-null  object 
 5   Name         31411 non-null  object 
 6   Item         31411 non-null  object 
 7   Qty          32089 non-null  float64
 8   Sales Price  31410 non-null  float64
 9   Amount       32089 non-null  float64
 10  Balance      32089 non-null  float64
dtypes: float64(5), object(6)
memory usage: 2.8+ MB

📈 Primeras 10 filas:


,Unnamed: 0,Type,Date,Num,Memo,Name,Item,Qty,Sales Price,Amount,Balance
0,ELLAS VELASQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Invoice,8/9/2025,19669.0,PAQUETE DE BOTECITOS 1 Litro,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 1 L PAQ,10.0,85.20,852.00,852.00
2,NaN,Invoice,8/9/2025,19669.0,PAQUETE BOTECITOS 1/2 LITRO,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 0.5 L PAQ,6.0,75.00,450.00,1302.00
3,NaN,Invoice,8/9/2025,19669.0,PAQUETE BOTECITOS 1.5 LITRO,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 1.5 L PAQ,6.0,95.40,572.40,1874.40
4,NaN,Invoice,8/9/2025,19669.0,IMPUESTO SOBRE VENTAS,ELLAS VELASQUEZ,ISV-15%,1.0,281.16,281.16,2155.56
5,Total ELLAS VELASQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,2155.56,2155.56
6,DELVIN LOZANO ROMERO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Sales Receipt,27/8/2025,19717.0,AGUA EN BOLSA 1/2,DELVIN LOZANO ROMERO,01 AGUA EN BOLSA:PaqAgua 1/2,80.0,12.00,960.00,960.00
8,NaN,Sales Receipt,27/8/2025,19717.0,SACO HIELO VACIO,DELVIN LOZANO ROMERO,04 HIELO:SacoVacio,8.0,12.00,96.00,1056.00
9,NaN,Sales Receipt,27/8/2025,19717.0,IMPUESTO SOBRE VENTAS,DELVIN LOZANO ROMERO,ISV-15%,1.0,0.00,0.00,1056.00


In [4]:
# Análisis más detallado de la estructura
print("🔎 ANÁLISIS DETALLADO DE LA ESTRUCTURA")
print("="*50)

# Verificar valores únicos en columnas clave
print(f"\n📊 Tipos de transacciones: {df['Type'].value_counts().to_dict()}")
print(f"\n👥 Número de clientes únicos: {df['Name'].nunique()}")
print(f"\n🛒 Número de productos únicos: {df['Item'].nunique()}")

# Mostrar algunos ejemplos de datos
print(f"\n💰 Estadísticas de montos:")
print(df['Amount'].describe())

print(f"\n📦 Estadísticas de cantidades:")
print(df['Qty'].describe())

# Verificar datos nulos
print(f"\n❓ Valores nulos por columna:")
print(df.isnull().sum())

# Mostrar algunos productos únicos
print(f"\n🛍️ Ejemplos de productos:")
productos_unicos = df['Item'].dropna().unique()[:10]
for i, producto in enumerate(productos_unicos, 1):
    print(f"{i}. {producto}")

# Mostrar algunos clientes únicos
print(f"\n👤 Ejemplos de clientes:")
clientes_unicos = df['Name'].dropna().unique()[:10]
for i, cliente in enumerate(clientes_unicos, 1):
    print(f"{i}. {cliente}")

🔎 ANÁLISIS DETALLADO DE LA ESTRUCTURA

📊 Tipos de transacciones: {'Sales Receipt': 23037, 'Invoice': 8374}

👥 Número de clientes únicos: 664

🛒 Número de productos únicos: 27

💰 Estadísticas de montos:
count     32089.000000
mean        488.220664
std        6678.086023
min           0.000000
25%          19.570000
50%          60.000000
75%         165.000000
max      742650.000000
Name: Amount, dtype: float64

📦 Estadísticas de cantidades:
count    32089.000000
mean        15.746053
std        184.692683
min          0.000000
25%          1.000000
50%          1.000000
75%          5.000000
max      14963.000000
Name: Qty, dtype: float64

❓ Valores nulos por columna:
Unnamed: 0     31411
Type            1357
Date            1357
Num             1357
Memo            1357
Name            1357
Item            1357
Qty              679
Sales Price     1358
Amount           679
Balance          679
dtype: int64

🛍️ Ejemplos de productos:
1. 02 BOTECITOS PAQUETES:BOTE 1 L PAQ
2. 02 BOTECIT

## 3. Limpieza y Preprocesamiento de Datos

In [ ]:
# Limpieza y preprocesamiento de datos
print("🧹 LIMPIEZA Y PREPROCESAMIENTO DE DATOS")
print("="*50)

# Crear una copia del dataframe original
df_original = df.copy()
print(f"📋 Datos originales: {len(df_original)} filas")

# 1. Filtrar solo las filas que corresponden a ventas reales (no totales ni encabezados)
# Las filas de ventas tienen Type no nulo y no son filas de totales
df_clean = df[
    (df['Type'].notna()) &  # Tiene tipo de transacción
    (~df['Type'].str.contains('Total', na=False)) &  # No es fila de total
    (df['Name'].notna()) &  # Tiene nombre de cliente
    (df['Item'].notna()) &  # Tiene producto
    (df['Amount'].notna())  # Tiene monto
].copy()

print(f"✅ Datos filtrados: {len(df_clean)} filas")
print(f"📉 Filas removidas: {len(df_original) - len(df_clean)} ({((len(df_original) - len(df_clean))/len(df_original)*100):.1f}%)")

# 2. Limpiar y convertir tipos de datos
print("\n🔧 Convirtiendo tipos de datos...")

# Convertir fechas
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d/%m/%Y', errors='coerce')

# Limpiar y convertir columnas numéricas
df_clean['Qty'] = pd.to_numeric(df_clean['Qty'], errors='coerce')
df_clean['Sales Price'] = pd.to_numeric(df_clean['Sales Price'], errors='coerce')
df_clean['Amount'] = pd.to_numeric(df_clean['Amount'], errors='coerce')
df_clean['Balance'] = pd.to_numeric(df_clean['Balance'], errors='coerce')

# Limpiar nombres de clientes y productos
df_clean['Name'] = df_clean['Name'].str.strip().str.upper()
df_clean['Item'] = df_clean['Item'].str.strip()

# 3. Remover filas con problemas en datos críticos
before_final_clean = len(df_clean)
df_clean = df_clean.dropna(subset=['Date', 'Amount', 'Name'])
print(f"📊 Después de remover filas con datos críticos faltantes: {len(df_clean)} filas")
print(f"🗑️ Filas removidas en limpieza final: {before_final_clean - len(df_clean)}")

# 4. Agregar columnas derivadas útiles
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month
df_clean['Day'] = df_clean['Date'].dt.day
df_clean['Weekday'] = df_clean['Date'].dt.day_name()
df_clean['Week'] = df_clean['Date'].dt.isocalendar().week

print(f"\n✅ LIMPIEZA COMPLETADA")
print(f"📈 Dataset final: {len(df_clean)} filas x {len(df_clean.columns)} columnas")
print(f"📅 Rango de fechas: {df_clean['Date'].min()} a {df_clean['Date'].max()}")
print(f"💰 Rango de montos: L {df_clean['Amount'].min():,.2f} a L {df_clean['Amount'].max():,.2f}")

# Mostrar una muestra de los datos limpios
print(f"\n📋 Muestra de datos limpios:")
df_clean.head()

🧹 LIMPIEZA Y PREPROCESAMIENTO DE DATOS
📋 Datos originales: 32768 filas
✅ Datos filtrados: 31411 filas
📉 Filas removidas: 1357 (4.1%)

🔧 Convirtiendo tipos de datos...
📊 Después de remover filas con datos críticos faltantes: 31411 filas
🗑️ Filas removidas en limpieza final: 0

✅ LIMPIEZA COMPLETADA
📈 Dataset final: 31411 filas x 16 columnas
📅 Rango de fechas: 2025-01-01 00:00:00 a 2025-09-11 00:00:00
💰 Rango de montos: $0.00 a $48,699.00

📋 Muestra de datos limpios:


,Unnamed: 0,Type,Date,Num,Memo,Name,Item,Qty,Sales Price,Amount,Balance,Year,Month,Day,Weekday,Week
1,NaN,Invoice,2025-09-08,19669.0,PAQUETE DE BOTECITOS 1 Litro,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 1 L PAQ,10.0,85.20,852.00,852.00,2025,9,8,Monday,37
2,NaN,Invoice,2025-09-08,19669.0,PAQUETE BOTECITOS 1/2 LITRO,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 0.5 L PAQ,6.0,75.00,450.00,1302.00,2025,9,8,Monday,37
3,NaN,Invoice,2025-09-08,19669.0,PAQUETE BOTECITOS 1.5 LITRO,ELLAS VELASQUEZ,02 BOTECITOS PAQUETES:BOTE 1.5 L PAQ,6.0,95.40,572.40,1874.40,2025,9,8,Monday,37
4,NaN,Invoice,2025-09-08,19669.0,IMPUESTO SOBRE VENTAS,ELLAS VELASQUEZ,ISV-15%,1.0,281.16,281.16,2155.56,2025,9,8,Monday,37
7,NaN,Sales Receipt,2025-08-27,19717.0,AGUA EN BOLSA 1/2,DELVIN LOZANO ROMERO,01 AGUA EN BOLSA:PaqAgua 1/2,80.0,12.00,960.00,960.00,2025,8,27,Wednesday,35


## 4. Métricas Clave de Negocio

In [ ]:
# Calcular métricas clave de negocio
print("📊 MÉTRICAS CLAVE DE NEGOCIO")
print("="*50)

# KPIs principales
total_ventas = df_clean['Amount'].sum()
total_transacciones = len(df_clean)
ticket_promedio = df_clean['Amount'].mean()
clientes_unicos = df_clean['Name'].nunique()
productos_unicos = df_clean['Item'].nunique()

print(f"💰 VENTAS TOTALES: L {total_ventas:,.2f}")
print(f"🛒 TOTAL TRANSACCIONES: {total_transacciones:,}")
print(f"🎯 TICKET PROMEDIO: L {ticket_promedio:.2f}")
print(f"👥 CLIENTES ÚNICOS: {clientes_unicos}")
print(f"📦 PRODUCTOS ÚNICOS: {productos_unicos}")

# Métricas por período
print(f"\n📅 ANÁLISIS POR PERÍODO:")
ventas_por_mes = df_clean.groupby(['Year', 'Month'])['Amount'].sum().reset_index()
ventas_por_mes['Periodo'] = ventas_por_mes['Year'].astype(str) + '-' + ventas_por_mes['Month'].astype(str).str.zfill(2)

print(f"📈 Ventas por mes:")
for _, row in ventas_por_mes.iterrows():
    print(f"   {row['Periodo']}: L {row['Amount']:,.2f}")

# Top clientes por valor
print(f"\n👑 TOP 5 CLIENTES POR VALOR:")
top_clientes = df_clean.groupby('Name')['Amount'].sum().sort_values(ascending=False).head()
for i, (cliente, valor) in enumerate(top_clientes.items(), 1):
    print(f"{i}. {cliente}: L {valor:,.2f}")

# Top productos por ventas
print(f"\n🏆 TOP 5 PRODUCTOS POR INGRESOS:")
top_productos_valor = df_clean.groupby('Item')['Amount'].sum().sort_values(ascending=False).head()
for i, (producto, valor) in enumerate(top_productos_valor.items(), 1):
    print(f"{i}. {producto}: L {valor:,.2f}")

print(f"\n📦 TOP 5 PRODUCTOS POR CANTIDAD:")
top_productos_qty = df_clean.groupby('Item')['Qty'].sum().sort_values(ascending=False).head()
for i, (producto, qty) in enumerate(top_productos_qty.items(), 1):
    print(f"{i}. {producto}: {qty:,.0f} unidades")

# Análisis por día de la semana
print(f"\n📅 VENTAS POR DÍA DE LA SEMANA:")
ventas_weekday = df_clean.groupby('Weekday')['Amount'].sum().sort_values(ascending=False)
for dia, valor in ventas_weekday.items():
    print(f"   {dia}: L {valor:,.2f}")

# Estadísticas adicionales
print(f"\n📊 ESTADÍSTICAS ADICIONALES:")
print(f"🔄 Transacciones por cliente promedio: {total_transacciones/clientes_unicos:.1f}")
print(f"📊 Valor promedio por cliente: L {total_ventas/clientes_unicos:.2f}")
print(f"🎯 Días con ventas: {df_clean['Date'].nunique()}")
print(f"📈 Ventas promedio por día: L {total_ventas/df_clean['Date'].nunique():.2f}")

📊 MÉTRICAS CLAVE DE NEGOCIO
💰 VENTAS TOTALES: $7,173,828.55
🛒 TOTAL TRANSACCIONES: 31,411
🎯 TICKET PROMEDIO: $228.39
👥 CLIENTES ÚNICOS: 664
📦 PRODUCTOS ÚNICOS: 27

📅 ANÁLISIS POR PERÍODO:
📈 Ventas por mes:
   2025-01: $756,616.14
   2025-02: $704,859.60
   2025-03: $806,289.49
   2025-04: $889,062.33
   2025-05: $964,247.79
   2025-06: $693,447.43
   2025-07: $814,829.26
   2025-08: $1,022,417.67
   2025-09: $522,058.84

👑 TOP 5 CLIENTES POR VALOR:
1. DURACRETO SA DE CV: $742,650.00
2. IDALIA ZELAYA: $276,835.16
3. NORTH OCEANICA S DE RL:INVERSIONES CANGREJAL: $248,447.50
4. DENNIE BUSH: $245,378.65
5. AZUL INTEREXPORT: $209,481.50

🏆 TOP 5 PRODUCTOS POR INGRESOS:
1. 03 BOTELLONES:Botellon Liquido: $1,281,869.00
2. 04 HIELO:Bolsa 5.5 Lbs: $1,091,645.10
3. 04 HIELO:Tonelada Escarcha: $974,495.34
4. ISV-15%: $723,076.65
5. 02 BOTECITOS PAQUETES:BOTE 1 L PAQ: $634,996.72

📦 TOP 5 PRODUCTOS POR CANTIDAD:
1. 04 HIELO:Bolsa 5.5 Lbs: 98,665 unidades
2. 03 BOTELLONES:Botellon Liquido: 59,454 u

## 5. Análisis de Ventas por Tiempo

In [ ]:
# Crear visualizaciones de series temporales
print("📈 CREANDO VISUALIZACIONES DE VENTAS POR TIEMPO")
print("="*50)

# 1. Ventas diarias
ventas_diarias = df_clean.groupby('Date').agg({
    'Amount': 'sum',
    'Qty': 'sum',
    'Name': 'nunique'  # Clientes únicos por día
}).reset_index()
ventas_diarias.columns = ['Fecha', 'Ventas', 'Cantidad', 'Clientes_Unicos']

# Gráfico de líneas de ventas diarias
fig_ventas_tiempo = px.line(
    ventas_diarias, 
    x='Fecha', 
    y='Ventas',
    title='💰 Evolución de Ventas Diarias - Honduras',
    labels={'Ventas': 'Ventas (L)', 'Fecha': 'Fecha'},
    line_shape='spline'
)
fig_ventas_tiempo.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Ventas (L)",
    hovermode='x unified',
    height=400
)
fig_ventas_tiempo.show()

# 2. Ventas por día de la semana (gráfico de barras)
ventas_weekday_df = df_clean.groupby('Weekday')['Amount'].sum().reset_index()
# Ordenar por días de la semana
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ventas_weekday_df['Weekday'] = pd.Categorical(ventas_weekday_df['Weekday'], categories=dias_orden, ordered=True)
ventas_weekday_df = ventas_weekday_df.sort_values('Weekday')

fig_weekday = px.bar(
    ventas_weekday_df,
    x='Weekday',
    y='Amount',
    title='📅 Ventas por Día de la Semana - Honduras',
    labels={'Amount': 'Ventas (L)', 'Weekday': 'Día de la Semana'},
    color='Amount',
    color_continuous_scale='viridis'
)
fig_weekday.update_layout(height=400)
fig_weekday.show()

# 3. Mapa de calor por día del mes y día de la semana
df_clean['Day_of_Month'] = df_clean['Date'].dt.day
heatmap_data = df_clean.groupby(['Weekday', 'Day_of_Month'])['Amount'].sum().reset_index()
heatmap_pivot = heatmap_data.pivot(index='Weekday', columns='Day_of_Month', values='Amount').fillna(0)

# Reordenar filas por días de la semana
heatmap_pivot = heatmap_pivot.reindex(dias_orden)

fig_heatmap = px.imshow(
    heatmap_pivot,
    title='🗓️ Mapa de Calor: Ventas por Día del Mes - Honduras',
    labels=dict(x="Día del Mes", y="Día de la Semana", color="Ventas (L)"),
    aspect="auto",
    color_continuous_scale='viridis'
)
fig_heatmap.update_layout(height=400)
fig_heatmap.show()

print("✅ Visualizaciones de tiempo creadas exitosamente!")

📈 CREANDO VISUALIZACIONES DE VENTAS POR TIEMPO


✅ Visualizaciones de tiempo creadas exitosamente!


## 6. Análisis de Productos Top

In [ ]:
# Análisis de productos más vendidos
print("🏆 ANÁLISIS DE PRODUCTOS TOP")
print("="*50)

# Top 10 productos por ingresos
top_productos_ingresos = df_clean.groupby('Item').agg({
    'Amount': 'sum',
    'Qty': 'sum'
}).sort_values('Amount', ascending=False).head(10).reset_index()

# Gráfico de barras horizontal para top productos por ingresos
fig_productos_ingresos = px.bar(
    top_productos_ingresos,
    x='Amount',
    y='Item',
    orientation='h',
    title='🏆 Top 10 Productos por Ingresos - Honduras',
    labels={'Amount': 'Ingresos (L)', 'Item': 'Producto'},
    color='Amount',
    color_continuous_scale='viridis'
)
fig_productos_ingresos.update_layout(
    height=500,
    yaxis={'categoryorder': 'total ascending'}
)
fig_productos_ingresos.show()

# Top 10 productos por cantidad vendida
top_productos_cantidad = df_clean.groupby('Item').agg({
    'Qty': 'sum',
    'Amount': 'sum'
}).sort_values('Qty', ascending=False).head(10).reset_index()

fig_productos_cantidad = px.bar(
    top_productos_cantidad,
    x='Qty',
    y='Item',
    orientation='h',
    title='📦 Top 10 Productos por Cantidad Vendida',
    labels={'Qty': 'Cantidad Vendida', 'Item': 'Producto'},
    color='Qty',
    color_continuous_scale='plasma'
)
fig_productos_cantidad.update_layout(
    height=500,
    yaxis={'categoryorder': 'total ascending'}
)
fig_productos_cantidad.show()

# Análisis de categorías de productos
print("\n🏷️ ANÁLISIS POR CATEGORÍAS DE PRODUCTOS:")

# Extraer categorías de productos basándose en palabras clave
def categorizar_producto(item):
    item_lower = str(item).lower()
    if 'agua' in item_lower or 'bolsa' in item_lower:
        return 'Agua en Bolsa'
    elif 'hielo' in item_lower:
        return 'Hielo'
    elif 'botellon' in item_lower:
        return 'Botellones'
    elif 'botecito' in item_lower or 'bote' in item_lower:
        return 'Botecitos'
    elif 'impuesto' in item_lower or 'isv' in item_lower:
        return 'Impuestos'
    else:
        return 'Otros'

df_clean['Categoria'] = df_clean['Item'].apply(categorizar_producto)

# Ventas por categoría
ventas_categoria = df_clean.groupby('Categoria').agg({
    'Amount': 'sum',
    'Qty': 'sum'
}).reset_index()

# Gráfico de pastel para distribución de ventas por categoría
fig_categoria_pie = px.pie(
    ventas_categoria,
    values='Amount',
    names='Categoria',
    title='🥧 Distribución de Ventas por Categoría de Producto',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig_categoria_pie.show()

# Gráfico de barras comparativo por categoría
fig_categoria_bar = px.bar(
    ventas_categoria,
    x='Categoria',
    y='Amount',
    title='📊 Ingresos por Categoría de Producto - Honduras',
    labels={'Amount': 'Ingresos (L)', 'Categoria': 'Categoría'},
    color='Amount',
    color_continuous_scale='viridis'
)
fig_categoria_bar.update_layout(height=400)
fig_categoria_bar.show()

print("✅ Análisis de productos completado!")

🏆 ANÁLISIS DE PRODUCTOS TOP



🏷️ ANÁLISIS POR CATEGORÍAS DE PRODUCTOS:


✅ Análisis de productos completado!


## 7. Análisis de Clientes Top

In [ ]:
# Análisis de clientes más valiosos
print("👑 ANÁLISIS DE CLIENTES TOP")
print("="*50)

# Métricas por cliente
clientes_metricas = df_clean.groupby('Name').agg({
    'Amount': ['sum', 'mean', 'count'],
    'Qty': 'sum',
    'Date': ['min', 'max']
}).round(2)

# Aplanar los nombres de columnas
clientes_metricas.columns = ['Total_Ventas', 'Ticket_Promedio', 'Num_Transacciones', 'Total_Cantidad', 'Primera_Compra', 'Ultima_Compra']
clientes_metricas = clientes_metricas.reset_index()

# Calcular días como cliente
clientes_metricas['Dias_Como_Cliente'] = (clientes_metricas['Ultima_Compra'] - clientes_metricas['Primera_Compra']).dt.days + 1
clientes_metricas['Frecuencia_Compras'] = clientes_metricas['Num_Transacciones'] / clientes_metricas['Dias_Como_Cliente']

# Top 15 clientes por valor total
top_clientes_valor = clientes_metricas.nlargest(15, 'Total_Ventas')

# Gráfico de barras horizontal para top clientes
fig_clientes_valor = px.bar(
    top_clientes_valor,
    x='Total_Ventas',
    y='Name',
    orientation='h',
    title='👑 Top 15 Clientes por Valor Total de Compras - Honduras',
    labels={'Total_Ventas': 'Ventas Totales (L)', 'Name': 'Cliente'},
    color='Total_Ventas',
    color_continuous_scale='viridis'
)
fig_clientes_valor.update_layout(
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)
fig_clientes_valor.show()

# Análisis de frecuencia vs valor
fig_scatter_clientes = px.scatter(
    clientes_metricas,
    x='Num_Transacciones',
    y='Total_Ventas',
    size='Ticket_Promedio',
    hover_data=['Name', 'Frecuencia_Compras'],
    title='💎 Análisis de Clientes: Frecuencia vs Valor - Honduras',
    labels={
        'Num_Transacciones': 'Número de Transacciones',
        'Total_Ventas': 'Ventas Totales (L)',
        'Ticket_Promedio': 'Ticket Promedio (L)'
    },
    color='Ticket_Promedio',
    color_continuous_scale='viridis'
)
fig_scatter_clientes.update_layout(height=500)
fig_scatter_clientes.show()

# Top clientes por frecuencia de compras
top_clientes_frecuencia = clientes_metricas.nlargest(10, 'Num_Transacciones')

fig_clientes_frecuencia = px.bar(
    top_clientes_frecuencia,
    x='Num_Transacciones',
    y='Name',
    orientation='h',
    title='🔄 Top 10 Clientes por Frecuencia de Compras',
    labels={'Num_Transacciones': 'Número de Transacciones', 'Name': 'Cliente'},
    color='Num_Transacciones',
    color_continuous_scale='plasma'
)
fig_clientes_frecuencia.update_layout(
    height=500,
    yaxis={'categoryorder': 'total ascending'}
)
fig_clientes_frecuencia.show()

# Distribución de clientes por rangos de valor
clientes_metricas['Rango_Valor'] = pd.cut(
    clientes_metricas['Total_Ventas'],
    bins=[0, 100, 500, 1000, 2000, float('inf')],
    labels=['L 0-100', 'L 100-500', 'L 500-1K', 'L 1K-2K', 'L 2K+']
)

distribucion_clientes = clientes_metricas['Rango_Valor'].value_counts().reset_index()
distribucion_clientes.columns = ['Rango_Valor', 'Cantidad_Clientes']

fig_distribucion = px.bar(
    distribucion_clientes,
    x='Rango_Valor',
    y='Cantidad_Clientes',
    title='📊 Distribución de Clientes por Rango de Valor - Honduras',
    labels={'Cantidad_Clientes': 'Número de Clientes', 'Rango_Valor': 'Rango de Valor'},
    color='Cantidad_Clientes',
    color_continuous_scale='viridis'
)
fig_distribucion.update_layout(height=400)
fig_distribucion.show()

print("✅ Análisis de clientes completado!")

👑 ANÁLISIS DE CLIENTES TOP


✅ Análisis de clientes completado!


## 8. Dashboard Interactivo

In [ ]:
# Dashboard Interactivo Consolidado
print("🎯 DASHBOARD INTERACTIVO DE INTELIGENCIA DE NEGOCIOS")
print("="*60)

# Crear un dashboard con múltiples subgráficos
from plotly.subplots import make_subplots

# Preparar datos para el dashboard
dashboard_fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Ventas Diarias', 'Top 5 Productos por Ingresos', 
                   'Ventas por Día de la Semana', 'Top 5 Clientes',
                   'Distribución por Categoría', 'Métricas Clave'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"type": "pie"}, {"type": "table"}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Ventas diarias (línea)
dashboard_fig.add_trace(
    go.Scatter(x=ventas_diarias['Fecha'], y=ventas_diarias['Ventas'],
              mode='lines+markers', name='Ventas Diarias', line=dict(color='#1f77b4')),
    row=1, col=1
)

# 2. Top 5 productos por ingresos (barras)
top_5_productos = top_productos_ingresos.head(5)
dashboard_fig.add_trace(
    go.Bar(x=top_5_productos['Amount'], y=top_5_productos['Item'],
           orientation='h', name='Top Productos', marker_color='#ff7f0e'),
    row=1, col=2
)

# 3. Ventas por día de la semana (barras)
dashboard_fig.add_trace(
    go.Bar(x=ventas_weekday_df['Weekday'], y=ventas_weekday_df['Amount'],
           name='Ventas por Día', marker_color='#2ca02c'),
    row=2, col=1
)

# 4. Top 5 clientes (barras horizontales)
top_5_clientes = top_clientes_valor.head(5)
dashboard_fig.add_trace(
    go.Bar(x=top_5_clientes['Total_Ventas'], y=top_5_clientes['Name'],
           orientation='h', name='Top Clientes', marker_color='#d62728'),
    row=2, col=2
)

# 5. Distribución por categoría (pie)
dashboard_fig.add_trace(
    go.Pie(labels=ventas_categoria['Categoria'], values=ventas_categoria['Amount'],
           name="Categorías"),
    row=3, col=1
)

# 6. Tabla de métricas clave
metricas_tabla = [
    ['Métrica', 'Valor'],
    ['Ventas Totales', f'L {total_ventas:,.2f}'],
    ['Total Transacciones', f'{total_transacciones:,}'],
    ['Ticket Promedio', f'L {ticket_promedio:.2f}'],
    ['Clientes Únicos', f'{clientes_unicos}'],
    ['Productos Únicos', f'{productos_unicos}']
]

dashboard_fig.add_trace(
    go.Table(
        header=dict(values=metricas_tabla[0], fill_color='lightblue'),
        cells=dict(values=list(zip(*metricas_tabla[1:])), fill_color='lightgray')
    ),
    row=3, col=2
)

# Configurar el layout del dashboard
dashboard_fig.update_layout(
    height=1200,
    title_text="🎯 Dashboard de Inteligencia de Negocios - Honduras 2025",
    title_x=0.5,
    title_font_size=20,
    showlegend=False
)

# Actualizar ejes
dashboard_fig.update_xaxes(title_text="Fecha", row=1, col=1)
dashboard_fig.update_yaxes(title_text="Ventas (L)", row=1, col=1)

dashboard_fig.update_xaxes(title_text="Ingresos (L)", row=1, col=2)
dashboard_fig.update_yaxes(title_text="Producto", row=1, col=2)

dashboard_fig.update_xaxes(title_text="Día de la Semana", row=2, col=1)
dashboard_fig.update_yaxes(title_text="Ventas (L)", row=2, col=1)

dashboard_fig.update_xaxes(title_text="Ventas Totales (L)", row=2, col=2)
dashboard_fig.update_yaxes(title_text="Cliente", row=2, col=2)

dashboard_fig.show()

print("\n🎉 ¡Dashboard Interactivo Creado Exitosamente!")
print("="*60)

🎯 DASHBOARD INTERACTIVO DE INTELIGENCIA DE NEGOCIOS



🎉 ¡Dashboard Interactivo Creado Exitosamente!


## 📈 Resumen Ejecutivo y Recomendaciones - Honduras

Basándome en el análisis de los datos de ventas de tu negocio en Honduras, aquí están los **insights clave** y **recomendaciones** estratégicas:

In [ ]:
# Generar recomendaciones basadas en el análisis
print("💡 INSIGHTS Y RECOMENDACIONES DE NEGOCIO")
print("="*60)

# Análisis de rendimiento por categoría
print("\n🎯 INSIGHTS CLAVE:")
print("-" * 40)

# 1. Análisis de productos más rentables
categoria_performance = df_clean.groupby('Categoria').agg({
    'Amount': ['sum', 'mean'],
    'Qty': 'sum'
}).round(2)
categoria_performance.columns = ['Total_Ingresos', 'Ingreso_Promedio', 'Cantidad_Total']
categoria_performance = categoria_performance.reset_index().sort_values('Total_Ingresos', ascending=False)

print("📊 Rendimiento por categoría:")
for _, row in categoria_performance.iterrows():
    print(f"   {row['Categoria']}: L {row['Total_Ingresos']:,.2f} (Promedio: L {row['Ingreso_Promedio']:.2f})")

# 2. Análisis de estacionalidad
mes_performance = df_clean.groupby('Month')['Amount'].sum().sort_values(ascending=False)
mejor_mes = mes_performance.index[0]
peor_mes = mes_performance.index[-1]

print(f"\n📅 Mejor mes: {mejor_mes} (L {mes_performance.iloc[0]:,.2f})")
print(f"📅 Mes más bajo: {peor_mes} (L {mes_performance.iloc[-1]:,.2f})")

# 3. Análisis de días de la semana
mejor_dia = ventas_weekday.index[0]
peor_dia = ventas_weekday.index[-1]

print(f"🗓️ Mejor día de la semana: {mejor_dia} (L {ventas_weekday.iloc[0]:,.2f})")
print(f"🗓️ Día más bajo: {peor_dia} (L {ventas_weekday.iloc[-1]:,.2f})")

# 4. Análisis de clientes
cliente_promedio_compras = clientes_metricas['Num_Transacciones'].mean()
cliente_promedio_valor = clientes_metricas['Total_Ventas'].mean()

print(f"\n👥 Cliente promedio: {cliente_promedio_compras:.1f} compras, L {cliente_promedio_valor:.2f} valor total")

# 5. Recomendaciones estratégicas
print("\n💡 RECOMENDACIONES ESTRATÉGICAS:")
print("-" * 40)

print("🚀 OPORTUNIDADES DE CRECIMIENTO:")
print(f"   1. Impulsar ventas en {peor_dia}: Ofertas especiales o promociones")
print(f"   2. Aprovechar el éxito de {mejor_dia}: Ampliar inventario")
print(f"   3. Enfocarse en la categoría más rentable: {categoria_performance.iloc[0]['Categoria']}")

print("\n🎯 ESTRATEGIAS DE RETENCIÓN:")
clientes_alta_frecuencia = len(clientes_metricas[clientes_metricas['Num_Transacciones'] >= 5])
clientes_baja_frecuencia = len(clientes_metricas[clientes_metricas['Num_Transacciones'] == 1])

print(f"   1. {clientes_alta_frecuencia} clientes frecuentes: Programa de lealtad")
print(f"   2. {clientes_baja_frecuencia} clientes de una sola compra: Campaña de reactivación")

print("\n📊 OPTIMIZACIÓN DE INVENTARIO:")
top_3_productos = top_productos_ingresos.head(3)
print("   Priorizar stock de:")
for i, (_, row) in enumerate(top_3_productos.iterrows(), 1):
    print(f"   {i}. {row['Item']} (L {row['Amount']:,.2f} en ventas)")

print("\n🔄 PRÓXIMOS PASOS:")
print("   1. Implementar sistema de alertas para días de baja venta")
print("   2. Desarrollar promociones cruzadas entre categorías")
print("   3. Crear dashboard automatizado para monitoreo diario")
print("   4. Establecer metas de ventas basadas en datos históricos")
print("   5. Considerar factores estacionales típicos de Honduras")

print("\n✅ Análisis completado. ¡Utiliza estos insights para impulsar tu negocio en Honduras!")
print("="*60)

💡 INSIGHTS Y RECOMENDACIONES DE NEGOCIO

🎯 INSIGHTS CLAVE:
----------------------------------------
📊 Rendimiento por categoría:
   Hielo: $2,743,408.83 (Promedio: $401.49)
   Agua en Bolsa: $1,349,993.10 (Promedio: $257.24)
   Botellones: $1,335,269.00 (Promedio: $331.83)
   Botecitos: $1,007,165.23 (Promedio: $573.56)
   Impuestos: $723,076.65 (Promedio: $53.37)
   Otros: $14,915.74 (Promedio: $7457.87)

📅 Mejor mes: 8 ($1,022,417.67)
📅 Mes más bajo: 9 ($522,058.84)
🗓️ Mejor día de la semana: Monday ($1,346,573.59)
🗓️ Día más bajo: Sunday ($469,358.50)

👥 Cliente promedio: 47.3 compras, $10803.96 valor total

💡 RECOMENDACIONES ESTRATÉGICAS:
----------------------------------------
🚀 OPORTUNIDADES DE CRECIMIENTO:
   1. Impulsar ventas en Sunday: Ofertas especiales o promociones
   2. Aprovechar el éxito de Monday: Ampliar inventario
   3. Enfocarse en la categoría más rentable: Hielo

🎯 ESTRATEGIAS DE RETENCIÓN:
   1. 386 clientes frecuentes: Programa de lealtad
   2. 5 clientes de un